In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/online-retail-ii-uci/online_retail_II.xlsx


In [2]:
pip install openpyxl

     |████████████████████████████████| 244 kB 863 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


## Görev 1 (Veriyi anlama ve hazırlama)

1. Online Retail II excelindeki 2010-2011 verisini okuyunuz. Oluşturduğunuz dataframe’in kopyasını oluşturunuz.


In [3]:
from pathlib import Path
DATA_DIR = Path("/kaggle/input/online-retail-ii-uci/online_retail_II.xlsx")
#Çalışma için Year2010-2011 sheeti kullanılacak!
dataset_ = pd.read_excel(DATA_DIR, sheet_name="Year 2010-2011")

2. Veri setinin betimsel istatistiklerini inceleyiniz.

In [4]:
dataset = dataset_.copy()

In [5]:
dataset.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
dataset.tail()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France
541909,581587,POST,POSTAGE,1,2011-12-09 12:50:00,18.00,12680.0,France


In [7]:
dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541910.0,9.552234,218.080957,-80995.00,1.00,3.00,10.00,80995.0
Price,541910.0,4.611138,96.759765,-11062.06,1.25,2.08,4.13,38970.0
Customer ID,406830.0,15287.684160,1713.603074,12346.00,13953.00,15152.00,16791.00,18287.0


Quantity ve Price değişkenlerinin %75 ve max değerleri arasında çok fazla fark var bu normal değil.

In [8]:
dataset.shape

(541910, 8)

In [9]:
dataset.index

RangeIndex(start=0, stop=541910, step=1)

In [10]:
dataset.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

3. Veri setinde eksik gözlem var mı? Varsa hangi değişkende kaç tane eksik gözlem vardır?

In [11]:
dataset.isnull().values.any()

True

In [12]:
dataset.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

4. Eksik gözlemleri veri setinden çıkartınız. Çıkarma işleminde ‘inplace=True’ parametresini kullanınız.

In [13]:
dataset.dropna(inplace = True)

In [14]:
dataset.shape

(406830, 8)

5. Eşsiz ürün sayısı kaçtır?

In [15]:
dataset["Description"].nunique()

3896

6. Hangi üründen kaçar tane vardır?

In [16]:
dataset["Description"].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    2070
REGENCY CAKESTAND 3 TIER              1905
JUMBO BAG RED RETROSPOT               1662
ASSORTED COLOUR BIRD ORNAMENT         1418
PARTY BUNTING                         1416
                                      ... 
ENAMEL DINNER PLATE PANTRY               1
72 CAKE CASES VINTAGE CHRISTMAS          1
ASSORTED TUTTI FRUTTI ROUND BOX          1
PEG BAG APPLE DESIGN                     1
GIRLY PINK TOOL SET                      1
Name: Description, Length: 3896, dtype: int64

7. En çok sipariş edilen 5 ürünü çoktan aza doğru sıralayınız.

In [17]:
dataset.groupby("Description").agg({"Quantity": "max"}).sort_values("Quantity", ascending = False).head()

,Quantity
Description,
"PAPER CRAFT , LITTLE BIRDIE",80995
MEDIUM CERAMIC TOP STORAGE JAR,74215
ASSTD DESIGN 3D PAPER STICKERS,12540
WORLD WAR 2 GLIDERS ASSTD DESIGNS,4800
SMALL POPCORN HOLDER,4300


8. Faturalardaki ‘C’ iptal edilen işlemleri göstermektedir. İptal edilen işlemleri veri setinden çıkartınız.

In [18]:
dataset = dataset[~dataset["Invoice"].str.contains("C", na=False)]

In [19]:
dataset.shape

(397925, 8)

9. Fatura başına elde edilen toplam kazancı ifade eden ‘TotalPrice’ adında bir değişken oluşturunuz.

In [20]:
dataset["TotalPrice"] = dataset["Quantity"]*dataset["Price"]

In [21]:
dataset.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


## RFM
* Recency: Sıcaklıktır. Müşterinin sıcaklığıdır. Müşterinin en son kaç gün önce geldiğini gösterir.
* Frequency: Sıklıktır. Müşteri kaç tane alışveriş yaptı demektir.
* Monetary: Müşterinin bıraktığı toplam paradır.


In [22]:
dataset["InvoiceDate"].max()

Timestamp('2011-12-09 12:50:00')

In [23]:
today = dt.datetime(2011, 12, 11)
rfm = dataset.groupby("Customer ID").agg({"InvoiceDate": lambda d: today - d.max(), 
                                    "Invoice": lambda n: n.nunique(),
                                    "TotalPrice": lambda TotalPrice: TotalPrice.sum()})

In [24]:
rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.0,326 days 13:59:00,1,77183.60
12347.0,3 days 08:08:00,7,4310.00
12348.0,76 days 10:47:00,4,1797.24
12349.0,19 days 14:09:00,1,1757.55
12350.0,311 days 07:59:00,1,334.40


In [25]:
rfm.columns = ["recency", "frequency", "monetary"]

In [26]:
rfm.head()

,recency,frequency,monetary
Customer ID,,,
12346.0,326 days 13:59:00,1,77183.60
12347.0,3 days 08:08:00,7,4310.00
12348.0,76 days 10:47:00,4,1797.24
12349.0,19 days 14:09:00,1,1757.55
12350.0,311 days 07:59:00,1,334.40


In [27]:
rfm = rfm[rfm["monetary"] > 0]

## Görev 3
RFM skorlarının oluşturulması ve tek bir değişkene çevrilmesi

*  Recency, Frequency ve Monetary metriklerini qcut yardımı ile 1-5 arasında skorlara çeviriniz.
Bu skorları recency_score, frequency_score ve monetary_score olarak kaydediniz.
*  Oluşan 2 farklı değişkenin değerini tek bir değişken olarak ifade ediniz ve RFM_SCORE olarak kaydediniz.
* * Örneğin; Ayrı ayrı değişkenlerde sırasıyla 5, 2 olan recency_score, frequency_score skorlarını RFM_SCORE değişkeni isimlendirmesi ile oluşturunuz.
# DİKKAT! Monetary skoru dahil etmiyoruz.

In [28]:
rfm.head()

,recency,frequency,monetary
Customer ID,,,
12346.0,326 days 13:59:00,1,77183.60
12347.0,3 days 08:08:00,7,4310.00
12348.0,76 days 10:47:00,4,1797.24
12349.0,19 days 14:09:00,1,1757.55
12350.0,311 days 07:59:00,1,334.40


In [29]:
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels = ([5,4,3,2,1]))

In [30]:
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

In [31]:
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels = ([1,2,3,4,5]))

In [32]:
rfm.head(10)

,recency,frequency,monetary,recency_score,frequency_score,monetary_score
Customer ID,,,,,,
12346.0,326 days 13:59:00,1,77183.60,1,1,5
12347.0,3 days 08:08:00,7,4310.00,5,5,5
12348.0,76 days 10:47:00,4,1797.24,2,4,4
12349.0,19 days 14:09:00,1,1757.55,4,1,4
12350.0,311 days 07:59:00,1,334.40,1,1,2
12352.0,37 days 09:23:00,8,2506.04,3,5,5
12353.0,205 days 06:13:00,1,89.00,1,1,1
12354.0,233 days 10:49:00,1,1079.40,1,1,4
12355.0,215 days 10:11:00,1,459.40,1,1,2


In [33]:
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str))

In [34]:
rfm.head(8)

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12346.0,326 days 13:59:00,1,77183.60,1,1,5,11
12347.0,3 days 08:08:00,7,4310.00,5,5,5,55
12348.0,76 days 10:47:00,4,1797.24,2,4,4,24
12349.0,19 days 14:09:00,1,1757.55,4,1,4,41
12350.0,311 days 07:59:00,1,334.40,1,1,2,11
12352.0,37 days 09:23:00,8,2506.04,3,5,5,35
12353.0,205 days 06:13:00,1,89.00,1,1,1,11
12354.0,233 days 10:49:00,1,1079.40,1,1,4,11


## Görev 4
RFM skorlarının segment olarak tanımlanması
* Oluşturulan RFM skorların daha açıklanabilir olması için segment tanımlamaları
yapınız.


In [35]:
seg_description = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'     
}

In [36]:
rfm["segment"] = rfm["RFM_SCORE"].replace(seg_description, regex = True) 

In [37]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.0,326 days 13:59:00,1,77183.60,1,1,5,11,hibernating
12347.0,3 days 08:08:00,7,4310.00,5,5,5,55,champions
12348.0,76 days 10:47:00,4,1797.24,2,4,4,24,at_risk
12349.0,19 days 14:09:00,1,1757.55,4,1,4,41,promising
12350.0,311 days 07:59:00,1,334.40,1,1,2,11,hibernating


## Görev 5
* Önemli bulduğunuz 3 segmenti seçiniz. Bu üç segmenti;
* * Hem aksiyon kararları açısından
* * Hem de segmentlerin yapısı açısından (ortalama RFM değerleri) yorumlayınız.  
* "Loyal Customers" sınıfına ait customer ID'leri seçerek excel çıktısını alınız

In [38]:
rfm["segment"].head(15)

Customer ID
12346.0            hibernating
12347.0              champions
12348.0                at_risk
12349.0              promising
12350.0            hibernating
12352.0        loyal_customers
12353.0            hibernating
12354.0            hibernating
12355.0            hibernating
12356.0    potential_loyalists
12357.0         about_to_sleep
12358.0    potential_loyalists
12359.0        loyal_customers
12360.0         need_attention
12361.0            hibernating
Name: segment, dtype: object

In [39]:
rfm[rfm["segment"] == "need_attention"].head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12360.0,53 days 08:38:00,3,2662.06,3,3,5,33,need_attention
12372.0,72 days 11:48:00,3,1298.04,3,3,4,33,need_attention
12413.0,67 days 15:00:00,3,758.10,3,3,3,33,need_attention
12456.0,45 days 09:19:00,3,3181.04,3,3,5,33,need_attention
12536.0,44 days 09:40:00,3,12601.83,3,3,5,33,need_attention


In [40]:
df1 = rfm[rfm["segment"] == "need_attention"]

In [41]:
df1.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12360.0,53 days 08:38:00,3,2662.06,3,3,5,33,need_attention
12372.0,72 days 11:48:00,3,1298.04,3,3,4,33,need_attention
12413.0,67 days 15:00:00,3,758.10,3,3,3,33,need_attention
12456.0,45 days 09:19:00,3,3181.04,3,3,5,33,need_attention
12536.0,44 days 09:40:00,3,12601.83,3,3,5,33,need_attention


In [42]:
df2 = rfm[rfm["segment"] == "new_customers"]

In [43]:
df2.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12367.0,5 days 07:12:00,1,168.90,5,1,1,51,new_customers
12442.0,4 days 09:38:00,1,172.06,5,1,1,51,new_customers
12478.0,4 days 14:04:00,1,680.99,5,1,3,51,new_customers
12479.0,12 days 10:34:00,1,527.20,5,1,3,51,new_customers
12558.0,8 days 13:19:00,1,269.96,5,1,2,51,new_customers


In [44]:
df3 = rfm[rfm["segment"] == "cant_loose"]

In [45]:
frames = [df1,df2,df3]

In [46]:
result = pd.concat(frames)

In [47]:
result.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12360.0,53 days 08:38:00,3,2662.06,3,3,5,33,need_attention
12372.0,72 days 11:48:00,3,1298.04,3,3,4,33,need_attention
12413.0,67 days 15:00:00,3,758.10,3,3,3,33,need_attention
12456.0,45 days 09:19:00,3,3181.04,3,3,5,33,need_attention
12536.0,44 days 09:40:00,3,12601.83,3,3,5,33,need_attention


In [48]:
#result.reset_index(inplace = True)

In [49]:
result.shape

(294, 8)

In [50]:
result.columns

Index(['recency', 'frequency', 'monetary', 'recency_score', 'frequency_score',
       'monetary_score', 'RFM_SCORE', 'segment'],
      dtype='object')

In [51]:
result.head(10)

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12360.0,53 days 08:38:00,3,2662.06,3,3,5,33,need_attention
12372.0,72 days 11:48:00,3,1298.04,3,3,4,33,need_attention
12413.0,67 days 15:00:00,3,758.10,3,3,3,33,need_attention
12456.0,45 days 09:19:00,3,3181.04,3,3,5,33,need_attention
12536.0,44 days 09:40:00,3,12601.83,3,3,5,33,need_attention
12577.0,36 days 10:36:00,3,657.80,3,3,3,33,need_attention
12759.0,53 days 09:11:00,3,1531.44,3,3,4,33,need_attention
12808.0,37 days 13:55:00,3,483.65,3,3,2,33,need_attention
12879.0,45 days 11:36:00,3,573.22,3,3,3,33,need_attention


In [52]:
result.tail(10)

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
17848.0,82 days 07:11:00,9,901.20,2,5,3,25,cant_loose
17850.0,373 days 08:33:00,34,5391.21,1,5,5,15,cant_loose
17874.0,186 days 11:06:00,6,683.46,1,5,3,15,cant_loose
17894.0,72 days 14:37:00,8,1119.81,2,5,4,25,cant_loose
17997.0,75 days 07:47:00,8,1970.67,2,5,4,25,cant_loose
18037.0,155 days 10:21:00,6,70.02,2,5,1,25,cant_loose
18073.0,115 days 09:58:00,12,3877.52,2,5,5,25,cant_loose
18094.0,82 days 14:32:00,7,3017.30,2,5,5,25,cant_loose
18168.0,177 days 04:39:00,6,1169.94,2,5,4,25,cant_loose


In [53]:
  loyal_customers_id = pd.DataFrame(rfm[rfm["segment"] == "loyal_customers"].index)

In [54]:
loyal_customers_id.to_excel("loyal_customers.xlsx",sheet_name = "loyalists")